In [1]:
import chseg
import numpy as np
import tensorflow as tf

from sklearn.metrics import classification_report

In [2]:
params = {
    'seq_len': 50,
    'batch_size': 128,
    'n_class': 4,
    'hidden_dim': 128,
    'clip_norm': 5.0,
    'text_iter_step': 10,
    'lr': {'start': 5e-3, 'end': 5e-4}
}

In [3]:
def to_test_seq(*args):
    return [np.reshape(x[:(len(x)-len(x)%params['seq_len'])],
        [-1,params['seq_len']]) for x in args]

def iter_seq(x):
    return np.array([x[i: i+params['seq_len']] for i in range(
        0, len(x)-params['seq_len'], params['text_iter_step'])])

def to_train_seq(*args):
    return [iter_seq(x) for x in args]

In [4]:
def rnn_cell():
    return tf.nn.rnn_cell.GRUCell(params['hidden_dim'],
        kernel_initializer=tf.orthogonal_initializer())

def clip_grads(loss):
    variables = tf.trainable_variables()
    grads = tf.gradients(loss, variables)
    clipped_grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    return zip(clipped_grads, variables)

def forward(x, reuse, is_training):
    with tf.variable_scope('model', reuse=reuse):
        x = tf.contrib.layers.embed_sequence(x, params['vocab_size'], params['hidden_dim'])
        x = tf.layers.dropout(x, 0.1, training=is_training)
        
        bi_outputs, _ = tf.nn.bidirectional_dynamic_rnn(
            rnn_cell(), rnn_cell(), x, dtype=tf.float32)
        x = tf.concat(bi_outputs, -1)
        
        logits = tf.layers.dense(x, params['n_class'])
    return logits

def model_fn(features, labels, mode, params):
    logits_tr = forward(features, reuse=False, is_training=True)
    logits_te = forward(features, reuse=True, is_training=False)
    seq_lens = tf.count_nonzero(features, 1)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        log_likelihood, _ = tf.contrib.crf.crf_log_likelihood(
            logits_tr, labels, seq_lens)
        
        loss_op = tf.reduce_mean(-log_likelihood)
        
        global_step = tf.train.get_global_step()
        
        lr_op = tf.train.exponential_decay(
            params['lr']['start'], global_step, params['lr']['steps'],
            params['lr']['end']/params['lr']['start'])
        
        train_op = tf.train.AdamOptimizer(lr_op).apply_gradients(
            clip_grads(loss_op), global_step=global_step)
        
        lth = tf.train.LoggingTensorHook({'lr': lr_op}, every_n_iter=100)
        
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss_op, train_op=train_op, training_hooks=[lth])
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        trans_params = tf.get_variable('transitions', [params['n_class'], params['n_class']])
        
        viterbi_seq, _ = tf.contrib.crf.crf_decode(
            logits_te, trans_params, seq_lens)
        
        return tf.estimator.EstimatorSpec(mode, predictions=viterbi_seq)

In [5]:
x_train, y_train, x_test, y_test, params['vocab_size'], word2idx, idx2word = chseg.load_data()
X_train, Y_train = to_train_seq(x_train, y_train)
X_test, Y_test = to_test_seq(x_test, y_test)
params['lr']['steps'] = len(X_train) // params['batch_size']

estimator = tf.estimator.Estimator(model_fn, params=params)

estimator.train(tf.estimator.inputs.numpy_input_fn(
    X_train, Y_train,
    batch_size = params['batch_size'],
    shuffle = True))

Y_pred = np.concatenate(list(estimator.predict(tf.estimator.inputs.numpy_input_fn(
    X_test,
    batch_size = params['batch_size'],
    shuffle = False))), 0)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp1p92fi01', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1096e7860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp1p92fi01/model.ckpt.
INFO:tensorflow:loss = 76.13855, step = 1
INFO:tensorflow:lr = 0.005
INFO:tensorflow:global_step/sec: 4.98011
INFO:tensorflow:loss = 12.126303, step = 101 (20.081 sec)
INFO:tensorflow:lr = 0.004086997 (20.081 sec)
INFO:tensorflow:global_step/sec: 5.02581
INFO:tensorflow:loss = 12.68087, step = 201 (19.897 sec)
INFO:tensorflow:lr = 0.0033407088 (19.897 sec)
INFO:tensorflow:global_step/sec: 4.97778
INFO:tensorflow:loss = 6.054343, step = 301 (20.089 sec)
INFO:tensorflow:lr = 0.0027306937 (20.089 sec)
INFO:tensorflow:global_step/sec: 5.12745
INFO:tensorflow:loss = 7.5701323, step = 401 (19.503 sec)
INFO:tensorflow:lr = 0.0022320673 (19.503 sec)
INFO:tensorflow:global_step/

In [6]:
print(classification_report(Y_test.ravel(), Y_pred.ravel(), target_names=['B','M','E','S']))

sample = '我来到大学读书，希望学到知识'
labels = list(estimator.predict(tf.estimator.inputs.numpy_input_fn(
    np.atleast_2d([word2idx[w] for w in sample] + [0]*(params['seq_len']-len(sample))),
    shuffle = False)))[0]

labels = labels[:len(sample)]
res = ''
for i, l in enumerate(labels):
    c = sample[i]
    if l == 2 or l == 3:
        c += ' '
    res += c
print(res)

             precision    recall  f1-score   support

          B       0.92      0.94      0.93    116058
          M       0.82      0.75      0.78     25425
          E       0.92      0.95      0.93    116057
          S       0.94      0.90      0.92    106810

avg / total       0.92      0.92      0.92    364350

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp1p92fi01/model.ckpt-1143
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
我 来到 大学 读书 ， 希望 学 到 知识 
